<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [22]:
import pandas as pd
import psycopg2

In [24]:
connection = psycopg2.connect(
    dbname= DBNAME,
    user= USER,
    host= HOST,
    password= PASSWORD,
    port= PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [25]:
# текст запроса
query_3_1 = '''
    SELECT COUNT(*) 
    FROM vacancies
'''

In [26]:
# результат запроса
display('Количество вакансий в базе', pd.read_sql_query(query_3_1, connection))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\899211099.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display('Количество вакансий в базе', pd.read_sql_query(query_3_1, connection))


'Количество вакансий в базе'

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [27]:
# текст запроса
query_3_2 = '''
    SELECT COUNT(*) FROM employers
'''

In [28]:
# результат запроса
display('Количество работодателей в базе', pd.read_sql(query_3_2, connection))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\2026426324.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display('Количество работодателей в базе', pd.read_sql(query_3_2, connection))


'Количество работодателей в базе'

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [29]:
# текст запроса
query_3_3 = '''
    SELECT COUNT(*) FROM areas
'''

In [30]:
# результат запроса
display('Количество регионов в базе', pd.read_sql_query(query_3_3, connection))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\1733736173.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display('Количество регионов в базе', pd.read_sql_query(query_3_3, connection))


'Количество регионов в базе'

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [31]:
# текст запроса
query_3_4 = '''
    SELECT COUNT(*) FROM industries
'''

In [32]:
# результат запроса
display('Количество сфер деятельности в базе', pd.read_sql_query(query_3_4, connection))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\3316058195.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display('Количество сфер деятельности в базе', pd.read_sql_query(query_3_4, connection))


'Количество сфер деятельности в базе'

,count
0,294


***

__выводы по предварительному анализу данных__

В базе данных имеются сведения о 49197 вакансиях от 23501 работодателя в 1362 регионах, и число сфер деятельности - 294.

Условно на от 1 работодателя имеется 2 вакансии. 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [33]:
query_4_1 = '''
    SELECT 
        ar.name AS area,
        COUNT(vac.id) AS cnt
    FROM vacancies AS vac
        JOIN areas AS ar ON vac.area_id = ar.id
    GROUP BY ar.id
    ORDER BY COUNT(vac.id) DESC
'''

In [34]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
display('Количество вакансий по каждому региону', df)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\2044735008.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


'Количество вакансий по каждому региону'

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [35]:
query_4_2 = '''
    SELECT COUNT(*)
    FROM vacancies
    WHERE (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
'''

In [36]:
df = pd.read_sql_query(query_4_2, connection)
print('Число вакансий с указанием хотя бы одной из границ зарплатной вилки - {}'.format(df['count'][0]))

Число вакансий с указанием хотя бы одной из границ зарплатной вилки - 24073


C:\Users\User\AppData\Local\Temp\ipykernel_16800\3875901878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [37]:
query_4_3 = ''' 
    SELECT 
        'Среднее значение границ зарплатной вилки' AS description,
        ROUND(AVG(salary_from)) AS lower_bound,
        ROUND(AVG(salary_to)) AS upper_bound
    FROM vacancies
'''

In [38]:
df = pd.read_sql_query(query_4_3, connection)
display(df)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\511848361.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,description,lower_bound,upper_bound
0,Среднее значение границ зарплатной вилки,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [39]:
query_4_4 = ''' 
    SELECT 
        schedule AS schedule,
        employment AS employment,
        COUNT(id)
    FROM vacancies
    GROUP BY schedule, employment
    ORDER BY 3 DESC
'''

In [40]:
df = pd.read_sql_query(query_4_4, connection)
display('Количество вакансий по сочетаниям типа рабочего графика и типа трудоустройства',df)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\1959819974.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


'Количество вакансий по сочетаниям типа рабочего графика и типа трудоустройства'

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [41]:
query_4_5 = ''' 
    SELECT 
        experience,
        COUNT(id)
    FROM vacancies
    GROUP BY experience
    ORDER BY 2
'''

In [42]:
df = pd.read_sql_query(query_4_5, connection)
display('Количество вакансий по условиям требуегомо опыта',df)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\232201511.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


'Количество вакансий по условиям требуегомо опыта'

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

__выводы по детальному анализу вакансий__

В базе данных по анализу вакансий можно заключить следующее:

ТОП-5 по количеству вакансий в регионе - Москва, Санк-Петербург, Минск, Новосибирск, Алматы. 
При этом макисамльное значения количества вакансий в регионе 5333 (Москва), минимальное в топе 1892 (Алматы).

У 24073 вакансий указана хотя бы одно значение зарплатной вилки - это примерно половина от всех вакансий.

Значение зарплатной вилки можно определить в границах срдених значений нижней и верхней границы - 71065 и 110537, соответсвенно.

Наибольшее количество вакансий с боьшим отпрывом имеют сочетание типа рабочего графика и типа трудоустройства - "Полный день" и "Полная занятость". На втором месте по численности вакансий "Удаленная работа" и "Полная занятость".
При этом можем наблюдать, что в сочетаниях графика и трудоустройства с наибольшими количством вакансий по типу трудоустройства преобладает "Полная занятость". Из этого можно предположить, что имеется явный приоритет таких условий по занятости от работодателей.

Минимальные сочетания по количеству вакансий - "Вахтовый метод" и "Проектная работа", "Сменный график" и "Проектная работа". 
Полнотью отсутствуют сочетания "Вахтовый метод" и "Стажировка", "Вахтовый метод" и "Частичная занятость". Работа вахтовым методом исключает возможность частичной занятости. А тип занятости "Стажировка" исключается работадателями для вакансий с типом расписания "Вахтовый метод", возможно, из-за того, что такие вакансии предполагают трудную сложную работу, которая требует наличие опыта и работодатели заинтересованы в опытных кадрах на сложных позициях, на которых необходим опыт.
Из этого можно предположить, что такие сочетания либо явлются редкой работой и достаточно уникальной, либо работадатели мало заинтересованны в таких условиях.

В наибольшем количестве (чуть больше половины) вакансий требования к опыту - от 1 года до 3 лет. Это можно связать с тем, что для многих должностей имеется большая "текучка", и также большинство рабочих мест как правило имеют необходимость в специалистах с некоторым опытом. 
Большинству работодателей нужны, возможно, специалисты "среднего" звена или, возмоно, нужны работники на место с высокой спенью сменности кадров. 

Также второе по числености вакансий требование к опыту - от 3 до 6 лет, что подтверждает выше указаное и предолагает собой то, что следующие по количеству требуются специалисты с значительных опытом - руководители, топ-менеджеры, мастера и узкие специалисты с опытом.

На третьем месте по количетсву вакансий тебование к опыту - Без опыта работы. Это возможно связано с тем что, такое требование обычно относится к вакансиям для стажеров, обслуживающий/технический персонал и вакансий для рабочих мест, которые только формируются и являются не ответсвенными.

Минимальное количество вакансий с требование к опыту - Более 6 лет. Это скорее всего связано с тем, вакансии с таким требованием преполагают работу узкого специалиста с больших опытом работы и хорошими навыками, с очень ответсвенными и важными функциями. Как правило это очень редкие и важных позиции.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [43]:
query_5_1 = '''
    SELECT 
        emp.name, 
        COUNT(vac.id) as count
    FROM employers AS emp
        JOIN vacancies AS vac ON vac.employer_id = emp.id
    GROUP BY emp.id
    ORDER BY count DESC
'''

In [44]:
df = pd.read_sql_query(query_5_1, connection)
display('Количество вакансий по работодателям', df)
print('На первом месте по количеству вакансий "{}"'.format(df['name'][0]))
print('На пятом месте по количеству вакансий "{}"'.format(df['name'][4]))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\2902578068.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


'Количество вакансий по работодателям'

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14901,Евсеев Сергей Владимирович,1
14902,Кадровые технологии,1
14903,ЛОМБАРД 24,1
14904,Авто-Компонент ТД,1


На первом месте по количеству вакансий "Яндекс"
На пятом месте по количеству вакансий "Газпром нефть"


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [45]:
query_5_2 = '''
    SELECT 
        areas.name, 
        COUNT(DISTINCT e.id) AS count_emp,
        COUNT(DISTINCT v.id) AS count_vac
    FROM areas
        LEFT JOIN employers AS e ON e.area = areas.id
        LEFT JOIN vacancies AS v ON v.area_id = areas.id
    GROUP BY areas.id
    ORDER BY count_vac DESC, count_emp DESC
'''

In [46]:
df = pd.read_sql_query(query_5_2, connection)
display('Количество работодателей и вакансий по регионам', df)
print(
    'Регион без вакансий и с максимальным количеством работадателей: {}'.format(
    df[df['count_vac'] == 0].sort_values(by='count_emp', ascending=False)['name'].iloc[0])
)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\2948835000.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


'Количество работодателей и вакансий по регионам'

,name,count_emp,count_vac
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Новосибирск,573,2006
4,Алматы,721,1892
...,...,...,...
1357,Новый Хушет,0,0
1358,Октябрьское (Пригородный район),0,0
1359,Визинга,0,0
1360,Косулино (Свердловская область),0,0


Регион без вакансий и с максимальным количеством работадателей: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [47]:
query_5_3 = '''
    SELECT 
        e.name, 
        COUNT(DISTINCT v.area_id) AS cnt
    FROM employers AS e
        LEFT JOIN vacancies AS v ON v.employer_id = e.id
    GROUP BY e.id
    ORDER BY cnt DESC
'''

In [48]:
df = pd.read_sql_query(query_5_3, connection)
display('Количество регионов в которых опубликованы вакансии по работодателям', df)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\425413825.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


'Количество регионов в которых опубликованы вакансии по работодателям'

,name,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [49]:
query_5_4 = '''
    SELECT 
        COUNT(e.id)
    FROM employers AS e
        LEFT JOIN employers_industries AS ei ON ei.employer_id = e.id
    WHERE ei.industry_id IS NULL
'''

In [50]:
df = pd.read_sql_query(query_5_4, connection)
print('Количество работодателей, у которых не указана сфера деятельности - {}'.format(df['count'][0]))

Количество работодателей, у которых не указана сфера деятельности - 8419


C:\Users\User\AppData\Local\Temp\ipykernel_16800\3137144242.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [51]:
query_5_5 = '''
    SELECT 
        e.name
    FROM employers AS e
        LEFT JOIN employers_industries AS ei ON ei.employer_id = e.id
    GROUP BY e.id
    HAVING COUNT(ei.industry_id) = 4
    ORDER BY name 
    OFFSET 2 LIMIT 1
'''

In [52]:
df = pd.read_sql_query(query_5_5, connection)
display('Компания на третьем месте в алфавитном списке работодателей с 4 сферами деятельности',df)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\322173941.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


'Компания на третьем месте в алфавитном списке работодателей с 4 сферами деятельности'

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [53]:
query_5_6 = '''
    SELECT 
        COUNT(e.id)
    FROM employers AS e
        LEFT JOIN employers_industries AS ei ON ei.employer_id = e.id
            JOIN industries AS i ON i.id = ei.industry_id
    WHERE i.name = 'Разработка программного обеспечения'
    GROUP BY i.id
    ORDER BY i.name 
'''

In [54]:
df = pd.read_sql_query(query_5_6, connection)
print('Количество работодателей со сферой деятельности "Разработка программного обеспечения"  - {}'.format(df['count'][0]))

Количество работодателей со сферой деятельности "Разработка программного обеспечения"  - 3553


C:\Users\User\AppData\Local\Temp\ipykernel_16800\1731032983.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [55]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url) # Делаем запрос
page = BeautifulSoup(response.text, 'html.parser') # получаем "страницу"
table = page.find_all('table', class_="standard sortable") # Берем нужную таблицу в которой есть название городов миллионеров
list_tr = table[0].find_all('tr') # Формируем поиском список тегов строк таблицы
cities_million = [] 
for i in range(1, len(list_tr)): # В цикле проходим по списку "строк" таблицы 
    tds = list_tr[i].find_all('td') # Берем из "строки" все значения "столбцов"
    cities_million.append(tds[1].text) # Берем значения нужного столбца с названием города
    
for i in range(len(cities_million)): # В цикле убираем в строке лишние символы (ссылки на источник)
    if '[' in cities_million[i]:
        cities_million[i] = cities_million[i][:cities_million[i].find('[')]
print(cities_million)


['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [56]:
query_5_7 = f'''
    (SELECT 
        a.name AS name,
        COUNT(v.id) AS cnt
    FROM areas AS a
        JOIN vacancies AS v ON v.area_id = a.id
            JOIN employers AS e ON e.id = v.employer_id
    WHERE (e.name = 'Яндекс') and (a.name in ('{"','".join(cities_million)}'))
    GROUP BY a.id
    ORDER BY cnt )
    
    UNION
    
    SELECT 
        'total',
        SUM(cnt)
    FROM (
        SELECT 
            a.name AS name,
            COUNT(v.id) AS cnt
        FROM areas AS a
            JOIN vacancies AS v ON v.area_id = a.id
                JOIN employers AS e ON e.id = v.employer_id
        WHERE (e.name = 'Яндекс') and (a.name in ('{"','".join(cities_million)}'))
    GROUP BY a.id
    ORDER BY cnt 
    ) AS table1
    
    ORDER BY cnt
'''

In [57]:
df = pd.read_sql_query(query_5_7, connection)
display('Количество вакансий "Яндекс" по городам-миллионерам',df)

C:\Users\User\AppData\Local\Temp\ipykernel_16800\1976652211.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


'Количество вакансий "Яндекс" по городам-миллионерам'

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

__выводы по анализу работодателей__

ТОП-5 работодателей по количеству вакансий: Яндекс (1933), Ростелеком (491), Тинькофф (444), СБЕР (428), Газпром нефть (331). При этом Яндекс имеет большой отрыв в числу вакансий.

ТОП-5 регионов по количеству вакансий в них и количеству работодателей: Москва (5333, 5864), Санкт-Петербург	(2851, 2217), Минск (2112, 1115), Новосибирск (2006, 573), Алматы (1892, 721). При этом для региона Москва наблюдается, что при количестве вакансий 5333 имеется 5864 работодателей, то есть работодатей больше чем количество вакансий, что, может быть, связанно с большим количеством вакансий от отдельных компаний и отсутвием у других, или с тем, что у большинства комапаний 1 вакансия и у некоторого числа их нет. Для других регионов в ТОПе наблюдается обратное соотношение вакансий больше чем работодателей по региону. Самое большое соотношение среди ТОПа у Новосибирск вакансий примерно больше чем в 3 раза чем работодателей. 

По количеству раегионов в которых выставлены вакансии выделяются работодатели - Яндекс (181), Ростелеком (152), Спецремонт (116). У остальных работодателей количестов регионов, в которых высталвенны вакансии, меньше 100.

У 8419 работодателей не указана сфера деятельности. Это примерно 36 процентов работодателей. При не указанной сфере деятельности не известно чем занимается работодатель, и вакансии от этого работодателя могут быть не определены для анализа по сферам деятельности. А также это сказывается на том, что не известно подтвержденный ли это работодатель офиально, и отражают ли вакансии действительность.

У 1138 работодателя указано 4 сферы деятельности. А 2384 работодателей имеют 4 и более сфер деятельности.
Разработкой программного обеспечения занимаются 3553 работодателя.

Наибольшее количество вакансий от Яндекс в городах-миллионерах - 54 вакансии в Москве. Наименьшее количество вакансий от Яндекс в городах-миллионерах  - 21 вакансия в Омске. Суммарное количество вакансий от Яндекс в городах-миллионерах - 485 вакансий.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [58]:
query_6_1 = '''
    SELECT 
        COUNT(v.id) AS cnt
    FROM vacancies AS v
    WHERE (lower(name) LIKE '%data%') OR (lower(name) LIKE '%данн%')
'''

In [59]:
df = pd.read_sql_query(query_6_1, connection)
print('Количество вакансий, которые имеют отношения к данным - {}'.format(df['cnt'][0]))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\2851489437.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


Количество вакансий, которые имеют отношения к данным - 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [60]:
query_6_2 = '''
    SELECT 
        COUNT(*)
    FROM vacancies AS v
    WHERE ( (lower(name) LIKE '%data scientist%') OR
        (lower(name) LIKE '%data science%') OR
        ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%')) OR 
        (lower(name) LIKE '%исследователь данных%') OR 
        (lower(name) LIKE '%machine learning%') OR 
        (lower(name) LIKE '%машинн%обучен%%') ) 
        AND ( (lower(name) LIKE '%junior%') OR
        (experience LIKE 'Нет опыта') OR
        (employment LIKE 'Стажировка') )
'''

In [61]:
df = pd.read_sql_query(query_6_2, connection)
print('Количество вакансий для начинающих дата-сайентист - {}'.format(df['count'][0]))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\3772489113.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


Количество вакансий для начинающих дата-сайентист - 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [62]:
query_6_3 = '''
    SELECT 
        COUNT(*)
    FROM vacancies AS v
    WHERE ( (lower(name) LIKE '%data scientist%') OR
        (lower(name) LIKE '%data science%') OR
        ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%')) OR 
        (lower(name) LIKE '%исследователь данных%') OR 
        (lower(name) LIKE '%machine learning%') OR 
        (lower(name) LIKE '%машинн%обучен%%') ) 
        AND ((key_skills LIKE '%postgres%') OR (key_skills LIKE '%SQL%'))
'''

In [63]:
df = pd.read_sql_query(query_6_3, connection)
print('Количество вакансий для DS с ключевыми навыками "SQL" и "postgres" - {}'.format(df['count'][0]))

Количество вакансий для DS с ключевыми навыками "SQL" и "postgres" - 201


C:\Users\User\AppData\Local\Temp\ipykernel_16800\2926998429.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [64]:
query_6_4 = '''
    SELECT 
        COUNT(*)
    FROM vacancies AS v
    WHERE ( (lower(name) LIKE '%data scientist%') OR
        (lower(name) LIKE '%data science%') OR
        ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%')) OR 
        (lower(name) LIKE '%исследователь данных%') OR 
        (lower(name) LIKE '%machine learning%') OR 
        (lower(name) LIKE '%машинн%обучен%%') ) 
        AND (key_skills LIKE '%Python%')
'''

In [65]:
df = pd.read_sql_query(query_6_4, connection)
print('Количество вакансий для DS с ключевыми навыками "Python" - {}'.format(df['count'][0]))

Количество вакансий для DS с ключевыми навыками "Python" - 351


C:\Users\User\AppData\Local\Temp\ipykernel_16800\3698713778.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [66]:
query_6_5 = '''
    WITH x AS (SELECT 
        v.key_skills AS skills,
        LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, '\t', '')) AS length,
        LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, '\t', '')) + 1 AS quantity
    FROM vacancies AS v
    WHERE ( (lower(name) LIKE '%data scientist%') OR
        (lower(name) LIKE '%data science%') OR
        ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%')) OR 
        (lower(name) LIKE '%исследователь данных%') OR 
        (lower(name) LIKE '%machine learning%') OR 
        (lower(name) LIKE '%машинн%обучен%%') ) 
    ) 
    SELECT 
        ROUND(AVG(quantity), 2) AS count
    FROM x
'''

In [67]:
df = pd.read_sql_query(query_6_5, connection)
print('Среднее значение количества ключевых навыков в вакансия для DS: {}'.format(df['count'][0]))

C:\Users\User\AppData\Local\Temp\ipykernel_16800\903286329.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


Среднее значение количества ключевых навыков в вакансия для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [68]:
# запрос
# пометка - 24073 строк таблицы, где указано хотя бы одно из полей границ заработной вилки.
query_6_6 = '''
    SELECT salary_experience.experience AS experience, ROUND(AVG(salary_experience.value)) AS value
    FROM ((WITH table1 AS (
            SELECT * FROM vacancies AS v
            WHERE (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
            EXCEPT 
            SELECT * FROM vacancies AS v
            WHERE (salary_from IS NOT NULL) AND (salary_to IS NOT NULL) )
        SELECT 
            experience AS experience, coalesce(salary_from, 0) + coalesce(salary_to, 0) AS value
        FROM table1
        WHERE (lower(name) LIKE '%data scientist%') OR
            (lower(name) LIKE '%data science%') OR
            ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%')) OR 
            (lower(name) LIKE '%исследователь данных%') OR 
            (lower(name) LIKE '%machine learning%') OR 
            (lower(name) LIKE '%машинн%обучен%%') )
        
        UNION ALL
        
        (SELECT experience, (salary_from + salary_to)/2 FROM vacancies
        WHERE ((salary_from IS NOT NULL) AND (salary_to IS NOT NULL)) 
            AND ( (lower(name) LIKE '%data scientist%') OR
            (lower(name) LIKE '%data science%') OR
            ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%')) OR 
            (lower(name) LIKE '%исследователь данных%') OR 
            (lower(name) LIKE '%machine learning%') OR 
            (lower(name) LIKE '%машинн%обучен%%') ) 
        ) ) AS salary_experience
    GROUP BY salary_experience.experience
    ORDER BY 2
'''

In [69]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
display(df)
res = df[df['experience']=='От 3 до 6 лет']['value'].iloc[0]
print(f'Средняя зарплата для DS c опытом работ от 3 до 6 лет сотставляет {res} рублей.')

C:\Users\User\AppData\Local\Temp\ipykernel_16800\3479244638.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,value
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


Средняя зарплата для DS c опытом работ от 3 до 6 лет сотставляет 243115.0 рублей.


In [70]:
connection.close()

***

__выводы по предметному анализу__

Вакансии в количестве 1771 имеют отношения к данным. Из вакансий по дата-сайенс 51 походит для начинающего (нет опыта работы).

В 201 вакансии по дата-сайенс в требованиях указаны ключевые навыки, такие как "SQL" и "postgres"
В 351 вакансии по дата-сайенс в требованиях указаны ключевые навыки, такие как "Python"
Из чего в отношении к общему числу вакансий дата-сайенс можно сделать вывод, что такие ключевые навыки являются нужными, но не является необходимыми, так как упор делается на другие, и, возмно, для вакансий DS указываются больше частные конкретные специализированные навыки как ключевые.
В среднем в вакансиях по дата-сайенс в требованиях указвается примерно 6-7 ключевых навыков.

Минимальная средняя предлагаемая плата по вакансиям DS без опыта работы 74643. 	Cредняя предлагаемая заработная плата от 1 года до 3 лет - 139675. Максимальная средняя предлагаемая плата по вакансиям DS от 3 до 6 лет 243115.
Из чего можно сделать вывод, что вакансии по DS являются высоко оплачиваемыми.


# Общий вывод по проекту

Были проведены предварительный анализ, детальный анализ, анализ работодателей, предметный анализ.

В предварительном анализе установлено, что в базе данных имеются сведения о 49197 вакансиях от 23501 работодателя в 1362 регионах, и число сфер деятельности - 294.
Условно на от 1 работодателя имеется 2 вакансии. 

В детальном анализе выявлены ТОП регионов с по кол-ву вакансий - это крупные и центральные города: Москва, Санк-Петербург, Минск, Новосибирск, Алматы. 
У половины вакансий нет указания заработной вилки, которая в среднем составляет 71065 и 110537.
Чаще работодатель ищет на полный день с полной занятостью или просто с требованием по типу полная занятость. Нет вакансий с требованием вахтового метода и стажировки и вахтового метода и частичной занятости.
Наибольшее кол-во вакансий по требованию опыта для требования от 1 до 3 лет, меньше всего ищут специалистов с опытом больше 6 лет.

В анализе работодателей установлено, что ТОП-5 регионов по количеству работодателей совпадает с ТОП-5 регионов по количеству вакансий, в ТОПе регионов по кол-ву вакансий в Москва имеет кол-во работодателей больше чем кол-во вакансий.
В ТОПе работодателей по кол-ву вакансий превалируют ИТ, Телекоминикация и банки, что определяет топ работодателей с областью деятельности с интернет и ИТ-технологии. Из всех работодателей с отрывом наибольшее число вакансий выставлено от Яндекс - 1933 вакансии в  181 регионе, что является максимальным числом регионов среди работодателей, из них 485 вакансий в годорах миллионерах, из них 54 в Москве. 
У 36 % работодателей не указана сфера деятельнсти, возможно какая-то часть из этих работодателей встречается примерно из половины всех вакансий в которых нет заработной вилки.
У 2384 работодателей 4 и более сфер деятельности. Разработкой программного обеспечения занимаются 3553 работодателя.

В предметном анализе установлено, что вакансии дата-сайенс или имеющие отношения к данным являются достаточно востребованнными и хорошо оплачиваемыми, в среднем от 74 тыс.руб. до 243 тыс.руб. 
В вакансиях по данным в среднем требуется около 6-7 ключевых навыков, среди них есть требования по Python, SQL и postgre, но при этом явно необходимы другие конкретные клюичевые навыки.

Также можно было бы исселдовать в дальнейщем распределение вакансий по регионам по разным сферам, определение общей или частичной схемы отрослей по регионам. Также какая часть из вакансий по опыту работы имеет зарплатые вилки, какая часть в каких регионах.